## Transpose Function with Pyspark ✌️

![allo](../image/transpose.png)

In [13]:
# Function shows all columns exist in table
def showDf(df, percent=1.0, maxRows=0, maxColumns=0):
    if (df == None): return
    import pandas
    from IPython.display import display
    if (maxColumns == 0): maxColumns = len(df.columns)
    pandas.set_option('display.max_columns', maxColumns)
    pandas.set_option('display.encoding', 'UTF-8')
    if (percent < 1.0):
        import datetime
        now = datetime.datetime.now()
        seed = long(now.strftime("%H%M%S"))
        df = df.sample(False, percent, seed)
    if (maxRows == 0): maxRows = df.count()
    pandas.set_option('display.max_rows', maxRows)
    dfp = df.toPandas()
    display(dfp)

In [11]:

from pyspark.sql.functions import array, col, explode, struct, lit
## just for example 
df = sc.parallelize([('client_A', 'bike', '100$'), ('client_A', 'car', '14000$'),('client_B', 'hover', '100$'), ("client_B", 'moto', '1400$')]).toDF(["customer", "product", "price"])
## Transpose Function 
def transpose(df, by):

    # Filter dtypes and split into column names and type description
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))
    # Spark SQL supports only homogeneous columns
    assert len(set(dtypes)) == 1, "All columns have to be of the same type"
     # Create and explode an array of (column_name, column_value) structs
    kvs = explode(array([
      struct(lit(c).alias("key"), col(c).alias("val")) for c in cols
    ])).alias("kvs")
    return df.select(by + [kvs]).select(by + ["kvs.key", "kvs.val"])


new_df = transpose(df, ["customer"])## [A] is te column to pivot on it

In [12]:
## The moment of truth :-D
print("The original Df is :")
print("********************")
df.show()  ## Native spark Show
print("********************")
print("The transposed Df is :")
print("********************")
new_df.show()

The original Df is :
********************
+--------+-------+------+
|customer|product| price|
+--------+-------+------+
|client_A|   bike|  100$|
|client_A|    car|14000$|
|client_B|  hover|  100$|
|client_B|   moto| 1400$|
+--------+-------+------+

********************
The transposed Df is :
********************
+--------+-------+------+
|customer|    key|   val|
+--------+-------+------+
|client_A|product|  bike|
|client_A|  price|  100$|
|client_A|product|   car|
|client_A|  price|14000$|
|client_B|product| hover|
|client_B|  price|  100$|
|client_B|product|  moto|
|client_B|  price| 1400$|
+--------+-------+------+



In [16]:
## The moment of truth :-D
print("The original Df is :")
print("********************")
print("")
showDf(df,1.0) ## call the showDf function
print("********************")
print("The transposed Df is :")
print("********************")
showDf(new_df,1.0) ## call the showDf function


The original Df is :
********************



,customer,product,price
0,client_A,bike,100$
1,client_A,car,14000$
2,client_B,hover,100$
3,client_B,moto,1400$


********************
The transposed Df is :
********************


,customer,key,val
0,client_A,product,bike
1,client_A,price,100$
2,client_A,product,car
3,client_A,price,14000$
4,client_B,product,hover
5,client_B,price,100$
6,client_B,product,moto
7,client_B,price,1400$
